In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as mc

df_new = pd.read_excel('overalldata.xlsx', sheet_name='Sheet1')

exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

control_group = filtered_df[filtered_df['model'] == 'original_text']
other_groups = filtered_df[filtered_df['model'] != 'original_text']

combined_df = pd.concat([control_group, other_groups])

#  one-way ANOVA
model = ols('Q("Flesch Reading Ease") ~ C(model)', data=combined_df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)

print(anova_table)

# post-hoc analysis using Tukey's HSD test
comp = mc.pairwise_tukeyhsd(combined_df['Flesch Reading Ease'], combined_df['model'])

tukey_results = pd.DataFrame(data=comp.summary().data[1:], columns=comp.summary().data[0])

# Filter for comparisons with the control group 'original_text'
control_comparisons = tukey_results[tukey_results['group1'] == 'original_text']

# Sort by absolute mean difference to find the most similar models
control_comparisons_sorted = control_comparisons.sort_values(by='meandiff', key=abs)

print(control_comparisons_sorted.head())


                sum_sq     df         F        PR(>F)
C(model)   4398.763496   28.0  3.762974  6.159026e-09
Residual  11439.102644  274.0       NaN           NaN
            group1                         group2  meandiff  p-adj    lower  \
405  original_text  qwen-2-72b-instruct@deepinfra   -16.121    0.0 -25.1787   

      upper  reject  
405 -7.0632    True  


In [ ]:
import pandas as pd
import statsmodels.api as sm
from statsmodels.formula.api import ols

df_new = pd.read_excel('overalldata.xlsx', sheet_name='Sheet1')

exclude_models = [
    "codellama-70b-instruct@anyscale",
    "codellama-7b-instruct@together-ai",
    "command-r-plus@aws-bedrock",
    "deepseek-coder-33b-instruct@together-ai",
    "phi-3-medium-4k-instruct@deepinfra",
    "nemotron-4-340b-instruct@deepinfra",
    "codellama-13b-instruct@together-ai"
]

filtered_df = df_new[~df_new['model'].isin(exclude_models)]

#  a summary table for the 'Flesch Reading Ease' grouped by 'model'
summary_table = filtered_df.groupby('model')['Flesch Reading Ease'].describe()

print(summary_table)

# Define and fit the ANOVA model
model = ols('Q("Flesch Reading Ease") ~ C(model)', data=filtered_df).fit()

anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)


                                        count       mean       std    min  \
model                                                                       
claude-3-haiku@anthropic                 15.0  77.174667  7.697056  63.09   
claude-3-opus@anthropic                  15.0  75.615333  8.322188  56.79   
claude-3-sonnet@anthropic                 1.0  66.670000       NaN  66.67   
claude-3.5-sonnet@anthropic              15.0  78.171333  7.014159  69.92   
gemini-1.5-flash@vertex-ai               15.0  77.988667  6.291568  68.60   
gemini-1.5-pro@vertex-ai                 15.0  77.161333  4.992453  70.43   
gemma-2-9b-it@fireworks-ai               15.0  78.304667  7.477700  62.27   
gemma-2b-it@together-ai                  14.0  74.838571  7.094929  63.19   
gemma-7b-it@anyscale                      1.0  64.000000       NaN  64.00   
gpt-3.5-turbo@openai                     15.0  74.166000  6.298815  59.33   
gpt-4-turbo@openai                       15.0  79.054667  4.369576  70.94   

In [ ]:
pip install pandas xlsxwriter


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 4.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd

summary_table = filtered_df.groupby('model')['Flesch Reading Ease'].describe()
anova_df = pd.DataFrame(anova_table)

output_file_path = 'summary_and_anova_output_Flesch Reading Ease.xlsx'
with pd.ExcelWriter(output_file_path, engine='xlsxwriter') as writer:
    summary_table.to_excel(writer, sheet_name='Summary Statistics')
    anova_df.to_excel(writer, sheet_name='ANOVA Table')

print(f"Data has been saved to {output_file_path}")


Data has been saved to summary_and_anova_output_Flesch Reading Ease.xlsx


In [ ]:
import pandas as pd

anova_df = pd.DataFrame(anova_table)
output_file = 'anova_table_output_Flesch Reading Ease.xlsx'
anova_df.to_excel(output_file, index=True)

print(f"ANOVA table saved as {output_file}")


ANOVA table saved as anova_table_output_Flesch Reading Ease.xlsx
